In [1]:
import pandas as pd
import numpy as np 
import re

### Dataset from kaggle.com

In [3]:
df = pd.read_excel("full_data.xlsx")

In [4]:
df.head(2)

,Ceremony,Year,Class,CanonicalCategory,Category,NomId,Film,FilmId,Name,Nominees,NomineeIds,Winner,Detail,Note,Citation,MultifilmNomination
0,1,1927/28,Acting,ACTOR IN A LEADING ROLE,ACTOR,an0051251,The Noose,tt0019217,Richard Barthelmess,Richard Barthelmess,nm0001932,NaN,Nickie Elkins,NaN,NaN,1.0
1,1,1927/28,Acting,ACTOR IN A LEADING ROLE,ACTOR,an0051252,The Patent Leather Kid,tt0018253,Richard Barthelmess,Richard Barthelmess,nm0001932,NaN,The Patent Leather Kid,NaN,NaN,1.0


In [5]:
df.columns = df.columns.str.lower()
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x) 
df.isnull().sum()

C:\Users\franc\AppData\Local\Temp\ipykernel_9312\3473697737.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


ceremony                   0
year                       0
class                      0
canonicalcategory          0
category                   0
nomid                    532
film                    1261
filmid                  1261
name                    1185
nominees                 353
nomineeids               880
winner                  8538
detail                  8843
note                   11398
citation               10831
multifilmnomination    11974
dtype: int64

In [6]:
df.drop(columns=[
        'class',
        'ceremony',
        'nomid',
        'name', 
        'nominees', 
        'nomineeids', 
        'detail',
        'note', 
        'citation', 
        'multifilmnomination'
        ], inplace=True)

In [7]:
#eliminar aquellos años que tienen '/' para poder pasar a int y filtrar
df = df[~df['year'].str.contains(r'/', regex=True)]
#Convertir a int
df['year'] = df['year'].astype(int)
#filtramos los 10 ultimos años
df = df[df['year']>=2000]


In [8]:
df.head()

,year,canonicalcategory,category,film,filmid,winner
8677,2000,actor in a leading role,actor in a leading role,before night falls,tt0247196,NaN
8678,2000,actor in a leading role,actor in a leading role,gladiator,tt0172495,1.0
8679,2000,actor in a leading role,actor in a leading role,cast away,tt0162222,NaN
8680,2000,actor in a leading role,actor in a leading role,pollock,tt0183659,NaN
8681,2000,actor in a leading role,actor in a leading role,quills,tt0180073,NaN


### Filtar la categoria del premio 
- Quedarnos solo con las mejores peliculas.

In [9]:
#quedarnos solo con las nominadas a mejor pelicula, animada y extranjera.
film_categories = [
    'best picture',
    'animated feature film',
    'international feature film']

df = df[df['canonicalcategory'].isin(film_categories)]


In [10]:
df.head(5)

,year,canonicalcategory,category,film,filmid,winner
8732,2000,international feature film,foreign language film,amores perros,tt0245712,NaN
8733,2000,international feature film,foreign language film,"crouching tiger, hidden dragon",tt0190332,1.0
8734,2000,international feature film,foreign language film,divided we fall,tt0234288,NaN
8735,2000,international feature film,foreign language film,everybody famous!,tt0209037,NaN
8736,2000,international feature film,foreign language film,the taste of others,tt0216787,NaN


In [11]:
df.isna().sum()

year                   0
canonicalcategory      0
category               0
film                   0
filmid                 0
winner               346
dtype: int64

In [12]:
df['winner'].unique()

array([nan,  1.])

In [13]:
# Reemplazar valores NaN con 0 y convertir la columna a enteros
df['winner'] =df['winner'].fillna(0).astype(int)
df.head()

,year,canonicalcategory,category,film,filmid,winner
8732,2000,international feature film,foreign language film,amores perros,tt0245712,0
8733,2000,international feature film,foreign language film,"crouching tiger, hidden dragon",tt0190332,1
8734,2000,international feature film,foreign language film,divided we fall,tt0234288,0
8735,2000,international feature film,foreign language film,everybody famous!,tt0209037,0
8736,2000,international feature film,foreign language film,the taste of others,tt0216787,0


In [14]:
#convertir la columna 'winner' tipo object 
df['winner'] = df['winner'].astype('object')
#convertir 0 por no y 1 por yes en la columna winner
df['winner'] = df['winner'].replace({0: 'no', 1: 'yes'})

In [15]:
df.to_csv('kaggle_clean.csv',index=False)


In [16]:
df_kaggle = pd.read_csv('kaggle_clean.csv')

In [17]:
df_kaggle.columns

Index(['year', 'canonicalcategory', 'category', 'film', 'filmid', 'winner'], dtype='object')

### Obtener informacion de la API de OBDM

In [17]:
#cargamos el df de kaggle ya limpio para que las imdb_ids nos sirvan de referencia para encontrar la informacion en la api. 
df = pd.read_csv('kaggle_clean.csv')

In [18]:
import requests
import time
API_KEY = 'c251b987'
base_url = 'http://www.omdbapi.com/'

In [19]:

def obtener_rating(imdb_id):
    url = 'http://www.omdbapi.com/'
    params = {
        'apikey': API_KEY,
        'i': imdb_id
    }
    
    response = requests.get(url, params=params)
    
    print(f"Consultando información para el IMDb ID: {imdb_id}")
    
    if response.status_code == 200:
        print(f"Respuesta recibida con éxito para el Id de IMDb: {imdb_id}")
        data = response.json()
        
        imdb_rating = float(data.get('imdbRating')) if data.get('imdbRating') != 'N/A' else None
        title = data.get('Title')  # Título de la película
        runtime = data.get('Runtime')  # Duración de la película
        genre = data.get('Genre')  # Género(s)
        director = data.get('Director')  # Director(es)
        actors = data.get('Actors')  # Actores principales
        language = data.get('Language')  # Idioma(s)
        country = data.get('Country')  # País(es) de producción
        metascore = data.get('Metascore')  # Puntuación en Metacritic
        imdb_votes = data.get('imdbVotes')  # Número de votos en IMDb
        box_office = data.get('BoxOffice')  # Recaudación en taquilla
        production = data.get('Production')  # Compañía productora
        
        return {
            'filmid': imdb_id,
            'title': title,
            'runtime': runtime,
            'genre': genre,
            'director': director,
            'actors': actors,
            'language': language,
            'country': country,
            'imdbRating': imdb_rating,
            'metascore': metascore,
            'imdbVotes': imdb_votes
        }
    else:
        print(f"Error al obtener datos para el ID: {imdb_id}, código de estado: {response.status_code}")
        return None


In [20]:
#Prueba funcion obtener rating
obtener_rating('tt0245712')

Consultando información para el IMDb ID: tt0245712
Respuesta recibida con éxito para el Id de IMDb: tt0245712


{'filmid': 'tt0245712',
 'title': 'Amores Perros',
 'runtime': '154 min',
 'genre': 'Drama, Thriller',
 'director': 'Alejandro G. Iñárritu',
 'actors': 'Emilio Echevarría, Gael García Bernal, Goya Toledo',
 'language': 'Spanish',
 'country': 'Mexico',
 'imdbRating': 8.0,
 'metascore': '83',
 'imdbVotes': '261,019'}

In [21]:
def obtener_df_imdb(df,archivo_salida='df_imdb.csv'):
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    print(f"📡 Iniciando consulta de {total} títulos únicos a OMDb...")

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🔍 ({i}/{total}) Consultando IMDb ID: {imdb_id}")
        datos.append(obtener_rating(imdb_id))
        time.sleep(0.25)  # Pausa para evitar bloqueo por la API

    print("✅ Consulta completada. Generando DataFrame...")
    df_info = pd.DataFrame(datos)
    print(f"💾 Guardando resultados en el archivo: {archivo_salida}")
    df_info.to_csv(archivo_salida, index=False)
    return df_info

In [ ]:
df_imdb = obtener_df_imdb(df)

📡 Iniciando consulta de 407 títulos únicos a OMDb...
🔍 (1/407) Consultando IMDb ID: tt0245712
Consultando información para el IMDb ID: tt0245712
Respuesta recibida con éxito para el Id de IMDb: tt0245712
🔍 (2/407) Consultando IMDb ID: tt0190332
Consultando información para el IMDb ID: tt0190332
Respuesta recibida con éxito para el Id de IMDb: tt0190332
🔍 (3/407) Consultando IMDb ID: tt0234288
Consultando información para el IMDb ID: tt0234288
Respuesta recibida con éxito para el Id de IMDb: tt0234288
🔍 (4/407) Consultando IMDb ID: tt0209037
Consultando información para el IMDb ID: tt0209037
Respuesta recibida con éxito para el Id de IMDb: tt0209037
🔍 (5/407) Consultando IMDb ID: tt0216787
Consultando información para el IMDb ID: tt0216787
Respuesta recibida con éxito para el Id de IMDb: tt0216787
🔍 (6/407) Consultando IMDb ID: tt0241303
Consultando información para el IMDb ID: tt0241303
Respuesta recibida con éxito para el Id de IMDb: tt0241303
🔍 (7/407) Consultando IMDb ID: tt0195685


In [34]:
df_imdb = pd.read_csv('df_imdb.csv')

In [20]:
df_imdb.isna().sum()

filmid        0
title         0
runtime       0
genre         0
director      0
actors        1
language      2
country       0
imdbRating    0
metascore     4
imdbVotes     0
dtype: int64

In [21]:
df_imdb = df_imdb.applymap(lambda x: x.lower()if isinstance(x,str) else x)

C:\Users\franc\AppData\Local\Temp\ipykernel_10924\852849305.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_imdb = df_imdb.applymap(lambda x: x.lower()if isinstance(x,str) else x)


### Web Scraping de BoxOfficeMojo


#### Worlwide Boxoffice

In [ ]:
# Función para obtener recaudación internacional desde Box Office Mojo
def obtener_recaudacion_mundial(imdb_id):
    url = f'https://www.boxofficemojo.com/title/{imdb_id}/?ref_=bo_se_r_1'
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al acceder a la página para el Id de IMDb: {imdb_id} - {e}")
        return 'Error al acceder a la página'

    soup = BeautifulSoup(response.text, 'html.parser')
    print(f"✅ Respuesta recibida con éxito para el Id de IMDb: {imdb_id}")

    try:
        # Buscar todas las secciones de resumen de rendimiento
        performance_sections = soup.select('div.mojo-performance-summary-table div.a-section.a-spacing-none')
        for section in performance_sections:
            if 'Worldwide' in section.get_text():
                money_span = section.find('span', class_='money')
                if money_span:
                    return money_span.get_text(strip=True)
        return None
    except Exception as e:
        print(f"❌ Error procesando el HTML: {e}")
        return None


In [ ]:
#Test Funcion recaudacion internacional 
obtener_recaudacion_mundial('tt0848228')

✅ Respuesta recibida con éxito para el Id de IMDb: tt0848228


'$1,520,538,536'

In [ ]:
# Función para extraer IMDb ID, título y recaudación internacional
def obtener_recaudaciones_mundiales(df):
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🌍 ({i}/{total}) Consultando recaudación para IMDb ID: {imdb_id}")
        recaudacion = obtener_recaudacion_mundial(imdb_id)
        time.sleep(0.25)
        
        # Obtener el título correspondiente desde el DataFrame original
        titulo = df[df['filmid'] == imdb_id]['film'].values[0]

        # Guardar los datos en la lista
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'Worlwide boxoffice': recaudacion
        })

    # Crear un DataFrame con los resultados
    df_worldwide = pd.DataFrame(datos)
    return df_worldwide


In [ ]:
# Ejecutar la función para obtener los datos y generar un archivo CSV
df_recaudacion_mundial = obtener_recaudaciones_mundiales(df)

# Guardar los resultados en un archivo CSV separado
df_recaudacion_mundial.to_csv('worlwide_boxoffice.csv', index=False)

print('✅ Proceso completado. Datos guardados en worlwide_boxoffice.csv.')


🌍 (1/407) Consultando recaudación para IMDb ID: tt0245712
✅ Respuesta recibida con éxito para el Id de IMDb: tt0245712
🌍 (2/407) Consultando recaudación para IMDb ID: tt0190332
✅ Respuesta recibida con éxito para el Id de IMDb: tt0190332
🌍 (3/407) Consultando recaudación para IMDb ID: tt0234288
✅ Respuesta recibida con éxito para el Id de IMDb: tt0234288
🌍 (4/407) Consultando recaudación para IMDb ID: tt0209037
✅ Respuesta recibida con éxito para el Id de IMDb: tt0209037
🌍 (5/407) Consultando recaudación para IMDb ID: tt0216787
✅ Respuesta recibida con éxito para el Id de IMDb: tt0216787
🌍 (6/407) Consultando recaudación para IMDb ID: tt0241303
✅ Respuesta recibida con éxito para el Id de IMDb: tt0241303
🌍 (7/407) Consultando recaudación para IMDb ID: tt0195685
✅ Respuesta recibida con éxito para el Id de IMDb: tt0195685
🌍 (8/407) Consultando recaudación para IMDb ID: tt0172495
✅ Respuesta recibida con éxito para el Id de IMDb: tt0172495
🌍 (9/407) Consultando recaudación para IMDb ID: 

#### Domestic Boxoffice

In [ ]:
# Función para obtener recaudación domestic desde Box Office Mojo
def obtener_recaudacion_domestic(imdb_id):
    url = f'https://www.boxofficemojo.com/title/{imdb_id}/?ref_=bo_se_r_1'
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al acceder a la página para el Id de IMDb: {imdb_id} - {e}")
        return 'Error al acceder a la página'

    soup = BeautifulSoup(response.text, 'html.parser')
    print(f"✅ Respuesta recibida con éxito para el Id de IMDb: {imdb_id}")

    try:
        # Buscar todas las secciones de resumen de rendimiento
        performance_sections = soup.select('div.mojo-performance-summary-table div.a-section.a-spacing-none')
        for section in performance_sections:
            if 'Domestic' in section.get_text():
                money_span = section.find('span', class_='money')
                if money_span:
                    return money_span.get_text(strip=True)
        return None
    except Exception as e:
        print(f"❌ Error procesando el HTML: {e}")
        return None

In [ ]:
#Test Funcion  
obtener_recaudacion_domestic('tt0848228')

✅ Respuesta recibida con éxito para el Id de IMDb: tt0848228


'$623,357,910'

In [ ]:
# Función para extraer IMDb ID, título y recaudación doomestic
def obtener_recaudaciones_domestic(df):
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🌍 ({i}/{total}) Consultando recaudación para IMDb ID: {imdb_id}")
        recaudacion = obtener_recaudacion_domestic(imdb_id)
        time.sleep(0.25)
        
        # Obtener el título correspondiente desde el DataFrame original
        titulo = df[df['filmid'] == imdb_id]['film'].values[0]

        # Guardar los datos en la lista
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'domestic boxoffice': recaudacion
        })

    # Crear un DataFrame con los resultados
    df_domestic_boxoffice = pd.DataFrame(datos)
    return df_domestic_boxoffice

In [ ]:
# Ejecutar la función para obtener los datos y generar un archivo CSV
df_recaudacion_domestic = obtener_recaudaciones_domestic(df)

# Guardar los resultados en un archivo CSV separado
df_recaudacion_domestic.to_csv('domestic_boxoffice.csv', index=False)

print('✅ Proceso completado. Datos guardados en domestic_boxoffice.csv.')

🌍 (1/407) Consultando recaudación para IMDb ID: tt0245712
✅ Respuesta recibida con éxito para el Id de IMDb: tt0245712
🌍 (2/407) Consultando recaudación para IMDb ID: tt0190332
✅ Respuesta recibida con éxito para el Id de IMDb: tt0190332
🌍 (3/407) Consultando recaudación para IMDb ID: tt0234288
✅ Respuesta recibida con éxito para el Id de IMDb: tt0234288
🌍 (4/407) Consultando recaudación para IMDb ID: tt0209037
✅ Respuesta recibida con éxito para el Id de IMDb: tt0209037
🌍 (5/407) Consultando recaudación para IMDb ID: tt0216787
✅ Respuesta recibida con éxito para el Id de IMDb: tt0216787
🌍 (6/407) Consultando recaudación para IMDb ID: tt0241303
✅ Respuesta recibida con éxito para el Id de IMDb: tt0241303
🌍 (7/407) Consultando recaudación para IMDb ID: tt0195685
✅ Respuesta recibida con éxito para el Id de IMDb: tt0195685
🌍 (8/407) Consultando recaudación para IMDb ID: tt0172495
✅ Respuesta recibida con éxito para el Id de IMDb: tt0172495
🌍 (9/407) Consultando recaudación para IMDb ID: 

#### International Boxoffice

In [ ]:
# Función para obtener recaudación domestic desde Box Office Mojo
def obtener_recaudacion_internacional(imdb_id):
    url = f'https://www.boxofficemojo.com/title/{imdb_id}/?ref_=bo_se_r_1'
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al acceder a la página para el Id de IMDb: {imdb_id} - {e}")
        return 'Error al acceder a la página'

    soup = BeautifulSoup(response.text, 'html.parser')
    print(f"✅ Respuesta recibida con éxito para el Id de IMDb: {imdb_id}")

    try:
        # Buscar todas las secciones de resumen de rendimiento
        performance_sections = soup.select('div.mojo-performance-summary-table div.a-section.a-spacing-none')
        for section in performance_sections:
            if 'International' in section.get_text():
                money_span = section.find('span', class_='money')
                if money_span:
                    return money_span.get_text(strip=True)
        return None
    except Exception as e:
        print(f"❌ Error procesando el HTML: {e}")
        return None

In [ ]:
# Función para extraer IMDb ID, título y recaudación international
def obtener_recaudaciones_internacional(df):
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🌍 ({i}/{total}) Consultando recaudación para IMDb ID: {imdb_id}")
        recaudacion = obtener_recaudacion_internacional(imdb_id)
        time.sleep(0.25)
        
        # Obtener el título correspondiente desde el DataFrame original
        titulo = df[df['filmid'] == imdb_id]['film'].values[0]

        # Guardar los datos en la lista
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'international boxoffice': recaudacion
        })

    # Crear un DataFrame con los resultados
    df_international_boxoffice = pd.DataFrame(datos)
    return df_international_boxoffice

In [ ]:
# Ejecutar la función para obtener los datos y generar un archivo CSV
df_recaudacion_international = obtener_recaudaciones_internacional(df)

# Guardar los resultados en un archivo CSV separado
df_recaudacion_international.to_csv('international_boxoffice.csv', index=False)

print('✅ Proceso completado. Datos guardados en domestic_boxoffice.csv.')

🌍 (1/407) Consultando recaudación para IMDb ID: tt0245712
✅ Respuesta recibida con éxito para el Id de IMDb: tt0245712
🌍 (2/407) Consultando recaudación para IMDb ID: tt0190332
✅ Respuesta recibida con éxito para el Id de IMDb: tt0190332
🌍 (3/407) Consultando recaudación para IMDb ID: tt0234288
✅ Respuesta recibida con éxito para el Id de IMDb: tt0234288
🌍 (4/407) Consultando recaudación para IMDb ID: tt0209037
✅ Respuesta recibida con éxito para el Id de IMDb: tt0209037
🌍 (5/407) Consultando recaudación para IMDb ID: tt0216787
✅ Respuesta recibida con éxito para el Id de IMDb: tt0216787
🌍 (6/407) Consultando recaudación para IMDb ID: tt0241303
✅ Respuesta recibida con éxito para el Id de IMDb: tt0241303
🌍 (7/407) Consultando recaudación para IMDb ID: tt0195685
✅ Respuesta recibida con éxito para el Id de IMDb: tt0195685
🌍 (8/407) Consultando recaudación para IMDb ID: tt0172495
✅ Respuesta recibida con éxito para el Id de IMDb: tt0172495
🌍 (9/407) Consultando recaudación para IMDb ID: 

In [18]:
df_recaudacion_domestic = pd.read_csv('domestic_boxoffice.csv')
df_recaudacion_international = pd.read_csv('international_boxoffice.csv')
df_recaudacion_world = pd.read_csv('worldwide_boxoffice.csv')



In [19]:
# Eliminar la columna 'title' si existe
for df in [df_recaudacion_international, df_recaudacion_world]:
    if 'title' in df.columns:
        df.drop(columns='title', inplace=True)

# Establecer 'IMDb ID' como índice
df_recaudacion_domestic.set_index('IMDb ID', inplace=True)
df_recaudacion_international.set_index('IMDb ID', inplace=True)
df_recaudacion_world.set_index('IMDb ID', inplace=True)

# Unir todos los DataFrames en uno solo
df_recaudacion = df_recaudacion_domestic.join(
    [df_recaudacion_international, df_recaudacion_world], 
    how='outer'
).reset_index()

In [20]:
df_recaudacion.dtypes

IMDb ID                    object
title                      object
domestic boxoffice         object
international boxoffice    object
Worlwide boxoffice         object
dtype: object

In [21]:
# Lista de columnas a limpiar
columnas_recaudacion = ['domestic boxoffice', 'international boxoffice', 'Worlwide boxoffice']

# Limpiar los valores y convertir a enteros
for col in columnas_recaudacion:
    df_recaudacion[col] = (
        df_recaudacion[col]
        .replace(r'[\$,]', '', regex=True)
        .replace('', np.nan)
        .fillna(0)
        .astype(int)
    )

In [22]:
df_recaudacion['domestic boxoffice']=df_recaudacion['domestic boxoffice'].replace('$','').replace(',','')

In [23]:
df_recaudacion.head()

,IMDb ID,title,domestic boxoffice,international boxoffice,Worlwide boxoffice
0,tt0245712,amores perros,5408467,15500000,20908467
1,tt0190332,"crouching tiger, hidden dragon",128530421,85446864,213978518
2,tt0234288,divided we fall,1332586,713854,2046440
3,tt0209037,everybody famous!,81734,15619,97353
4,tt0216787,the taste of others,891369,130569,1021938


In [24]:
df_recaudacion.to_csv('boxoffice.csv', index=False)

In [25]:
df_boxoffice = pd.read_csv('boxoffice.csv')

In [26]:
df_boxoffice.head()

,IMDb ID,title,domestic boxoffice,international boxoffice,Worlwide boxoffice
0,tt0245712,amores perros,5408467,15500000,20908467
1,tt0190332,"crouching tiger, hidden dragon",128530421,85446864,213978518
2,tt0234288,divided we fall,1332586,713854,2046440
3,tt0209037,everybody famous!,81734,15619,97353
4,tt0216787,the taste of others,891369,130569,1021938


### Web scraping www.the-numbers.com

In [ ]:
def film_url_fixed(film_name):
    film_name = film_name.strip()

    if film_name.startswith("The "):
        film_name = film_name[4:] + ", The"
    elif film_name.startswith("A "):
        film_name = film_name[2:] + ", A"

    film_name = film_name.replace("&", "and")
    for char in [" ", ":", ",",  "?","/", "(", ")"]:
        film_name = film_name.replace(char, "-")
    for char1 in [".", "'", "!",]:
        film_name = film_name.replace(char1, "")
    return film_name

In [ ]:
def obtener_presupuesto(film_name):
    film_url = film_url_fixed(film_name)
    url = f'https://www.the-numbers.com/movie/{film_url}#tab=summary'

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        b_tags = soup.find_all('b', string='Production Budget:')
        for b in b_tags:
            td_parent = b.find_parent('td')
            if td_parent:
                td_siguiente = td_parent.find_next_sibling('td')
                if td_siguiente:
                    texto_completo = td_siguiente.text.strip()
                    texto_dinero = texto_completo.split(' (')[0]  # "$2,000,000"
                    # Limpiar y convertir a int
                    presupuesto = int(texto_dinero.replace('$', '').replace(',', ''))
                    print(f"   ✅ Presupuesto encontrado: {presupuesto}")
                    return presupuesto
        print("   ⚠️ Presupuesto no encontrado")
        return None
    else:
        print("   ❌ Error al acceder a la página")
        return None


In [ ]:
# Ejemplo de uso
print(obtener_presupuesto("Amores Perros"))

   ✅ Presupuesto encontrado: $2,000,000
$2,000,000


In [ ]:
# Función para obtener presupuestos de todas las películas en el DataFrame
def obtener_presupuestos(df):
    datos = []
    peliculas = df['title'].unique()
    total = len(peliculas)
    
    for i, movie_name in enumerate(peliculas, start=1):
        print(f"🎬 ({i}/{total}) Consultando presupuesto para: {movie_name}")
        presupuesto = obtener_presupuesto(movie_name)
        time.sleep(0.25)
        
        # Guardar los datos en la lista
        datos.append({
            'title': movie_name,
            'budget': presupuesto
        })

    # Crear un DataFrame con los resultados
    df_presupuestos = pd.DataFrame(datos)
    return df_presupuestos

In [ ]:
def obtener_presupuestos(df):
    datos = []
    ids = df['filmid'].unique()
    total = len(ids)

    for i, imdb_id in enumerate(ids, start=1):
        print(f"🎬 ({i}/{total}) Consultando presupuesto para IMDb ID: {imdb_id}")
        
        # Obtener el título correspondiente desde el DataFrame original
        titulo = df[df['filmid'] == imdb_id]['title'].values[0]

        # Obtener el presupuesto
        presupuesto = obtener_presupuesto(titulo)
        time.sleep(0.25)

        # Guardar los datos en la lista
        datos.append({
            'IMDb ID': imdb_id,
            'title': titulo,
            'budget': presupuesto
        })

    # Crear un DataFrame con los resultados
    df_presupuestos = pd.DataFrame(datos)
    return df_presupuestos

In [ ]:
df = pd.read_csv('df_imdb.csv')  # Cargar tu DataFrame
df_presupuestos = obtener_presupuestos(df)

# Guardar los resultados en un archivo CSV
df_presupuestos.to_csv('movie_budgets.csv', index=False)
print("✅ Datos guardados en 'movie_budgets.csv'")

🎬 (1/407) Consultando presupuesto para IMDb ID: tt0245712
   ✅ Presupuesto encontrado: 2000000
🎬 (2/407) Consultando presupuesto para IMDb ID: tt0190332
   ✅ Presupuesto encontrado: 15000000
🎬 (3/407) Consultando presupuesto para IMDb ID: tt0234288
   ⚠️ Presupuesto no encontrado
🎬 (4/407) Consultando presupuesto para IMDb ID: tt0209037
   ⚠️ Presupuesto no encontrado
🎬 (5/407) Consultando presupuesto para IMDb ID: tt0216787
   ⚠️ Presupuesto no encontrado
🎬 (6/407) Consultando presupuesto para IMDb ID: tt0241303
   ✅ Presupuesto encontrado: 25000000
🎬 (7/407) Consultando presupuesto para IMDb ID: tt0195685
   ✅ Presupuesto encontrado: 50000000
🎬 (8/407) Consultando presupuesto para IMDb ID: tt0172495
   ✅ Presupuesto encontrado: 103000000
🎬 (9/407) Consultando presupuesto para IMDb ID: tt0181865
   ✅ Presupuesto encontrado: 48000000
🎬 (10/407) Consultando presupuesto para IMDb ID: tt0268397
   ✅ Presupuesto encontrado: 25000000
🎬 (11/407) Consultando presupuesto para IMDb ID: tt019878

In [31]:
df_budget =pd.read_csv('movie_budgets.csv')


In [32]:
# Crear el diccionario con los datos de presupuesto
presupuestos = {
    'Amélie': 10000000,
    'Lagaan: Once Upon a Time in India': 9000000,
    'Spirited Away': 19200000,
    'The Crime of Padre Amaro': 3000000,
    'The Triplets of Belleville': 10000000,
    'The Barbarian Invasions': 4500000,
    'The Chorus': 13000000,
    'The Sea Inside': 11500000,
    'Yesterday': 26000000,
    'The Aviator': 190000000,
    'Howl\'s Moving Castle': 19000000,
    'Sophie Scholl: The Final Days': 6000000,
    'After the Wedding': 7000000,
    'Days of Glory': 20000000,
    'The Lives of Others': 2000000,
    'Pan\'s Labyrinth': 19000000,
    'The Departed': 90000000,
    'The Counterfeiters': 5000000,
    'Mongol: The Rise of Genghis Khan': 20000000,
    'WALL·E': 180000000,
    'The Baader Meinhof Complex': 23000000,
    'The Class': 7000000,
    'Departures': 4500000,
    'The Milk of Sorrow': 2000000,
    'A Prophet': 10000000,
    'The Secret in Their Eyes': 4000000,
    'The White Ribbon': 11000000,
    'The Blind Side': 29000000,
    'An Education': 7000000,
    'Precious': 10000000,
    'The Illusionist': 16000000,
    'In a Better World': 6000000,
    'The Fighter': 25000000,
    'The Social Network': 40000000,
    'A Cat in Paris': 6000000,
    'Bullhead': 3000000,
    'Footnote': 1500000,
    'A Separation': 3000000,
    'The Tree of Life': 32000000,
    'A Royal Affair': 10000000,
    'War Witch': 2000000,
    'Les Misérables': 61000000,
    'Ernest & Celestine': 8000000,
    'The Wind Rises': 35000000,
    'The Great Beauty': 13000000,
    'The Hunt': 4000000,
    'The Missing Picture': 1000000,
    'The Tale of The Princess Kaguya': 49000000,
    'Tangerines': 1500000,
    'Wild Tales': 3000000,
    'Birdman or (The Unexpected Virtue of Ignorance)': 18000000,
    'The Theory of Everything': 15000000,
    'The Boy and the World': 2000000,
    'Shaun the Sheep Movie': 25000000,
    'When Marnie Was There': 20000000,
    'Embrace of the Serpent': 1200000,
    'Son of Saul': 1500000,
    'A War': 5000000,
    'The Martian': 108000000,
    'The Revenant': 135000000,
    'My Life as a Zucchini': 8000000,
    'The Red Turtle': 6000000,
    'Land of Mine': 6000000,
    'A Man Called Ove': 7000000,
    'The Salesman': 2000000,
    'A Fantastic Woman': 2000000,
    'The Insult': 4000000,
    'Loveless': 5000000,
    'On Body and Soul': 3000000,
    'The Square': 10000000,
    'Darkest Hour': 30000000,
    'Lady Bird': 10000000,
    'Mirai': 9000000,
    'Spider-Man: Into the Spider-Verse': 90000000,
    'Capernaum': 4000000,
    'Never Look Away': 30000000,
    'Roma': 15000000,
    'Shoplifters': 3000000,
    'Bohemian Rhapsody': 52000000,
    'The Favourite': 15000000,
    'Green Book': 23000000,
    'A Star Is Born': 36000000,
    'How to Train Your Dragon: The Hidden World': 129000000,
    'I Lost My Body': 6000000,
    'Klaus': 40000000,
    'Missing Link': 100000000,
    'Honeyland': 500,
    'Pain and Glory': 11000000,
    'Parasite': 11400000,
    'The Irishman': 160000000,
    'Jojo Rabbit': 14000000,
    '1917': 95000000,
    'Onward': 200000000,
    'Over the Moon': 38000000,
    'A Shaun the Sheep Movie: Farmageddon': 25000000,
    'Soul': 150000000,
    'Wolfwalkers': 10000000,
    'Another Round': 4000000,
    'Better Days': 9000000,
    'Collective': 1000000,
    'The Man Who Sold His Skin': 1500000,
    'Quo Vadis, Aida?': 2300000,
    'The Father': 6000000,
    'Encanto': 120000000,
    'Flee': 3000000,
    'Luca': 50000000,
    'The Mitchells vs. the Machines': 85000000,
    'Drive My Car': 2000000,
    'The Hand of God': 5000000,
    'Lunana: A Yak in the Classroom': 200,
    'The Worst Person in the World': 5000000,
    'Belfast': 20000000,
    'CODA': 10000000,
    'Dune: Part One': 165000000,
    'King Richard': 50000000,
    'Licorice Pizza': 40000000,
    'The Power of the Dog': 39000000,
    'Guillermo del Toro\'s Pinocchio': 35000000,
    'Marcel the Shell with Shoes On': 6300000,
    'Puss in Boots: The Last Wish': 90000000,
    'The Sea Beast': 70000000,
    'Turning Red': 175000000,
    'Argentina, 1985': 10000000,
    'Close': 3000000,
    'EO': 2000000,
    'The Quiet Girl': 1000000,
    'Avatar: The Way of Water': 460000000,
    'The Banshees of Inisherin': 20000000,
    'Elvis': 85000000,
    'The Fabelmans': 40000000,
    'Tár': 35000000,
    'Top Gun: Maverick': 170000000,
    'Women Talking': 10000000,
    'The Boy and the Heron': 25000000,
    'Elemental': 175000000,
    'Robot Dreams': 50000000,
    'Spider-Man: Across the Spider-Verse': 100000000,
    'Io Capitano': 3000000,
    'Perfect Days': 4000000,
    'Society of the Snow': 10000000,
    'The Teachers\' Lounge': 4000000,
    'The Zone of Interest': 8000000,
    'American Fiction': 7000000,
    'Anatomy of a Fall': 8000000,
    'The Holdovers': 25000000,
    'Oppenheimer': 100000000,
    'Past Lives': 2000000,
    'Poor Things': 50000000,
    'Flow': 5000000,
    'Inside Out 2': 200000000,
    'Memoir of a Snail': 1000000,
    'Wallace & Gromit: Vengeance Most Fowl': 30000000,
    'The Wild Robot': 40000000,
    'Emilia Pérez': 3000000,
    'The Girl with the Needle': 2000000,
    'The Seed of the Sacred Fig': 1500000,
    'Anora': 2000000,
    'The Brutalist': 12000000,
    'A Complete Unknown': 2000000,
    'Conclave': 10000000,
    'Dune: Part Two': 200000000,
    'Nickel Boys': 4000000,
    'The Substance': 6000000
}

# Reemplazar los presupuestos en el DataFrame
df_budget['budget'] = df_budget['title'].map(presupuestos).fillna(df_budget['budget'])
df_budget['budget'] = df_budget['budget'].fillna(0)
# Guardar el archivo actualizado
df_budget.to_csv('movie_budgets_updated.csv', index=False)

print(df_budget.head())  # Para revisar los primeros registros

     IMDb ID                           title      budget
0  tt0245712                   Amores Perros   2000000.0
1  tt0190332  Crouching Tiger, Hidden Dragon  15000000.0
2  tt0234288                 Divided We Fall         0.0
3  tt0209037             Everybody's Famous!         0.0
4  tt0216787             The Taste of Others         0.0


In [27]:
df_budget = pd.read_csv('movie_budgets_updated.csv')

In [28]:
# Eliminar la columna 'title' si existe
for df in [df_budget]:
    if 'title' in df.columns:
        df.drop(columns='title', inplace=True)
        
df_budget.set_index('IMDb ID', inplace=True)
df_boxoffice.set_index('IMDb ID', inplace=True)

df_financial_info = df_boxoffice.join(
    df_budget, 
    how='inner'
).reset_index()


In [29]:
df_financial_info['ROI'] = (
    (df_financial_info['Worlwide boxoffice'] - df_financial_info['budget']) / df_financial_info['budget']
).round(2)



In [30]:
df_financial_info.head(10)

,IMDb ID,title,domestic boxoffice,international boxoffice,Worlwide boxoffice,budget,ROI
0,tt0245712,amores perros,5408467,15500000,20908467,2000000.0,9.45
1,tt0190332,"crouching tiger, hidden dragon",128530421,85446864,213978518,15000000.0,13.27
2,tt0234288,divided we fall,1332586,713854,2046440,0.0,inf
3,tt0209037,everybody famous!,81734,15619,97353,0.0,inf
4,tt0216787,the taste of others,891369,130569,1021938,0.0,inf
5,tt0241303,chocolat,71509363,81191263,152700626,25000000.0,5.11
6,tt0195685,erin brockovich,125595205,132256558,257851763,50000000.0,4.16
7,tt0172495,gladiator,187705427,277801273,465516248,103000000.0,3.52
8,tt0181865,traffic,124115725,83400000,207515725,48000000.0,3.32
9,tt0268397,jimmy neutron: boy genius,80936232,22056304,102992536,25000000.0,3.12


In [31]:
df_financial_info.to_csv('financial_info.csv',index=False)

In [32]:
df_financial_info=pd.read_csv('financial_info.csv')

In [35]:
# Eliminar columnas si existen
columnas_a_eliminar = {'title', 'domestic boxoffice', 'international boxoffice'}
if columnas_a_eliminar.issubset(df_financial_info.columns):
    df_financial_info.drop(columns=columnas_a_eliminar, inplace=True)



# Hacer inner join
df_final = df_imdb.join(df_financial_info, how='inner')

,Unnamed: 0,filmid,title,runtime,genre,director,actors,language,country,imdbRating,metascore,imdbVotes,IMDb ID,Worlwide boxoffice,budget,ROI,winner,category
0,0,tt0245712,amores perros,154 min,"drama, thriller",alejandro g. iñárritu,"emilio echevarría, gael garcía bernal, goya to...",spanish,mexico,8.0,83.0,"261,019",tt0245712,20908467,2000000.0,9.45,no,foreign language film
1,1,tt0190332,"crouching tiger, hidden dragon",120 min,"action, adventure, drama",ang lee,"chow yun-fat, michelle yeoh, ziyi zhang","mandarin, chinese","china, taiwan, hong kong, united states",7.9,94.0,"285,783",tt0190332,213978518,15000000.0,13.27,yes,foreign language film
2,1,tt0190332,"crouching tiger, hidden dragon",120 min,"action, adventure, drama",ang lee,"chow yun-fat, michelle yeoh, ziyi zhang","mandarin, chinese","china, taiwan, hong kong, united states",7.9,94.0,"285,783",tt0190332,213978518,15000000.0,13.27,no,best picture
3,2,tt0234288,divided we fall,117 min,"comedy, drama, war",jan hrebejk,"bolek polívka, csongor kassai, jaroslav dusek","german, czech",czech republic,7.6,69.0,"5,448",tt0234288,2046440,0.0,inf,no,foreign language film
4,3,tt0209037,everybody's famous!,97 min,"comedy, drama, music",dominique deruddere,"josse de pauw, eva van der gucht, werner de smedt","dutch, english, spanish","belgium, netherlands, france",6.9,60.0,"3,326",tt0209037,97353,0.0,inf,no,foreign language film


In [ ]:
df_final = df_final.join(df_kaggle.set_index('filmid')[['year','winner', 'category']], on='filmid', how='inner')



In [51]:
df_final.head(5)

,filmid,title,runtime,genre,director,actors,language,country,imdbRating,metascore,imdbVotes,IMDb ID,Worlwide boxoffice,budget,ROI,year,winner,category
0,tt0245712,Amores Perros,154 min,Drama,Alejandro G. Iñárritu,"Emilio Echevarría, Gael García Bernal, Goya To...",Spanish,Mexico,8.0,83.0,"261,019",tt0245712,20908467,2000000.0,9.45,2000,no,foreign language film
1,tt0190332,"Crouching Tiger, Hidden Dragon",120 min,Action,Ang Lee,"Chow Yun-Fat, Michelle Yeoh, Ziyi Zhang","Mandarin, Chinese",China,7.9,94.0,"285,783",tt0190332,213978518,15000000.0,13.27,2000,yes,foreign language film
1,tt0190332,"Crouching Tiger, Hidden Dragon",120 min,Action,Ang Lee,"Chow Yun-Fat, Michelle Yeoh, Ziyi Zhang","Mandarin, Chinese",China,7.9,94.0,"285,783",tt0190332,213978518,15000000.0,13.27,2000,no,best picture
2,tt0234288,Divided We Fall,117 min,Comedy,Jan Hrebejk,"Bolek Polívka, Csongor Kassai, Jaroslav Dusek","German, Czech",Czech Republic,7.6,69.0,"5,448",tt0234288,2046440,0.0,inf,2000,no,foreign language film
3,tt0209037,Everybody's Famous!,97 min,Comedy,Dominique Deruddere,"Josse De Pauw, Eva van der Gucht, Werner De Smedt","Dutch, English, Spanish",Belgium,6.9,60.0,"3,326",tt0209037,97353,0.0,inf,2000,no,foreign language film


In [52]:
df_final['genre'] = df_final['genre'].str.split(',').str[0].str.strip()
df_final['country'] = df_final['country'].str.split(',').str[0].str.strip()

In [53]:
df_final.to_csv('final_dataset.csv',index=False)


In [40]:
df = pd.read_csv('final_dataset.csv')

In [41]:
df.columns

Index(['filmid', 'title', 'runtime', 'genre', 'director', 'actors', 'language',
       'country', 'imdbRating', 'metascore', 'imdbVotes', 'IMDb ID',
       'Worlwide boxoffice', 'budget', 'ROI', 'year', 'winner', 'category'],
      dtype='object')